## Описание эксперимента:
Есть мобильное приложение. В этом приложении у пользователей есть возможность покупать игровые предметы за реальные деньги. Чтобы стимулировать пользователей их покупать, приложение периодически предлагает пользователям товары - появляется окошко с рекомендацией купить товар. Отдел машинного обучения предложил улучшение для текущего алгоритма выбора рекомендации. Для проверки улучшений алгоритма был проведен A/B тест. Лог его проведения предоставлен в прикрепленном файле. 

## Метрика: 
средний доход от пользователя за 1 неделю после первого показа ему рекомендации на 10% (после начала A/B теста время первого показа ищется снова)

## Важная информация:
Эксперимент начинается 2023-05-01. Данные есть до 2023-06-01 (но можно завершить раньше, если это позволит оценка длительности)
Вам сказали, что его длительность должна составить 1 месяц.
Все покупки, которые вызваны не влиянием рекомендаций, в этом логе не учитываются

## Описание данных:
- id_product -  идентификатор продукта, который был рекомендован
- is_pay - купил ли пользователь товар
- sum_payment - размер платежа (0, если не купил)
- city - город, в котором находится пользователь
- id_user - пользователь
- timestamp - timestamp события
- date - дата события

## Задачи, которые необходимо решить:
Оценить длительность теста на момент его начала. Сравнить с предложенной. Для оценки необходимо использовать данные с пред экспериментального периода. Посмотреть, есть ли выбросы в данных.
Построить методику расчета целевой метрики. Рассчитать целевую метрику на день окончания теста (рассчитанной в п1) для группы A и B, рассчитать эффект, p_value. Посмотреть, есть ли выбросы в данных.
Рассчитать метрики из п2 по дням и построить их графики.
Принять решение о результате теста - обосновать.

In [5]:
import pandas as pd

In [11]:
df = pd.read_csv("ab_made_4")

In [12]:
df

,timestamp,id_user,sum_payment,group,city,id_product,is_pay,date
0,1680330573,user_9903,27,NaN,Санкт-Петербург,4.0,1,2023-04-01
1,1680332652,user_6732,0,NaN,Рязань,1.0,0,2023-04-01
2,1680378039,user_4199,0,NaN,Москва,3.0,0,2023-04-01
3,1680337580,user_3606,12,NaN,Санкт-Петербург,7.0,1,2023-04-01
4,1680334389,user_9519,0,NaN,Санкт-Петербург,14.0,0,2023-04-01
...,...,...,...,...,...,...,...,...
56400,1685485266,user_10740,0,A,Санкт-Петербург,14.0,0,2023-05-31
56401,1685481632,user_3589,0,B,Санкт-Петербург,1.0,0,2023-05-31
56402,1685514383,user_10664,13,A,Москва,17.0,1,2023-05-31
56403,1685481325,user_3537,0,B,Ярославь,19.0,0,2023-05-31


In [7]:
!ls

HW01.ipynb HW02.ipynb HW04.ipynb ab_made_4
